## Run Monte Carlo one crypto at a time pulling 3 years of history from API

In [1]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [3]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [36]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [9]:
import datetime

In [28]:
today = date.today()
three_years_ago = date.today() - datetime.timedelta(days=3*365)
three_years_ago

datetime.date(2019, 1, 23)

In [25]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = ['AAPL']

# Set timeframe to 1D 
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp(three_years_ago, tz='America/New_York').isoformat()
end_date = pd.Timestamp(today, tz='America/New_York').isoformat()

In [26]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Review the first and last 5 rows of the Alpaca DataFrame
df_portfolio


AAPL                                    
                              open    high      low   close     volume
time                                                                  
2021-08-31 00:00:00-04:00  152.680  152.80  151.290  151.83   65181250
2021-09-01 00:00:00-04:00  152.800  154.98  152.340  152.54   72982461
2021-09-02 00:00:00-04:00  153.850  154.72  152.400  153.71   64119121
2021-09-03 00:00:00-04:00  153.750  154.63  153.090  154.31   51476040
2021-09-07 00:00:00-04:00  155.000  157.26  154.390  156.70   74486147
...                            ...     ...      ...     ...        ...
2022-01-14 00:00:00-05:00  171.340  173.78  171.090  173.05   68538760
2022-01-18 00:00:00-05:00  171.500  172.54  169.405  169.83   72755895
2022-01-19 00:00:00-05:00  169.980  171.08  165.940  166.21   80743311
2022-01-20 00:00:00-05:00  166.980  169.68  164.180  164.52   77228033
2022-01-21 00:00:00-05:00  164.415  166.33  162.300  162.35  107284910

[100 rows x 5 columns]

In [37]:
request_url = 'https://data.alpaca.markets/v1beta1/crypto/ETHUSD/bars?exchanges=CBSE&APCA-API-KEY-ID='+alpaca_api_key+'&APCA-API-SECRET-KEY='+alpaca_secret_key

In [40]:
alpaca_api_key

'PKRRZU02BBLADTM9NDFP'

In [38]:
# Execute the API call to Alpaca Crypto API using requests.get
response = requests.get(request_url)

In [39]:
response

<Response [403]>

In [ ]:
# Create an empty DataFrame for closing prices
closing_prices_df = pd.DataFrame()

# Get the closing prices of FB and TWTR
closing_prices_df["FB"] = df_portfolio["FB"]["close"]
closing_prices_df["TWTR"] = df_portfolio["TWTR"]["close"]

# Drop the time component of the date
closing_prices_df.index = closing_prices_df.index.date

# Display sample data
closing_prices_df.head(10)

## Attempt to use CoinGecko for crypto price history API

In [41]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [42]:
cg.get_price(ids='bitcoin', vs_currencies='usd')

{'bitcoin': {'usd': 35359}}

In [52]:
results = cg.get_coin_market_chart_range_by_id(id='bitcoin', vs_currency='usd', from_timestamp=1548294012, to_timestamp=1642988412)

In [53]:
df = pd.DataFrame(results)

In [54]:
df

,prices,market_caps,total_volumes
0,"[1548374400000, 3565.6420046170365]","[1548374400000, 62397665388.19675]","[1548374400000, 2514079887.672157]"
1,"[1548460800000, 3564.822417111422]","[1548460800000, 62386316357.47355]","[1548460800000, 2582590473.407218]"
2,"[1548547200000, 3563.614563419144]","[1548547200000, 62374968460.904976]","[1548547200000, 2548165746.089901]"
3,"[1548633600000, 3553.121681129778]","[1548633600000, 62192208387.693]","[1548633600000, 2882787006.773549]"
4,"[1548720000000, 3432.215668301249]","[1548720000000, 60087154448.40433]","[1548720000000, 3445780457.599724]"
...,...,...,...
1090,"[1642550400000, 42395.45879157343]","[1642550400000, 803962100417.9756]","[1642550400000, 18897950050.285973]"
1091,"[1642636800000, 41749.55143098559]","[1642636800000, 790507925308.7825]","[1642636800000, 19626113309.554142]"
1092,"[1642723200000, 40707.6824143097]","[1642723200000, 774235809673.4731]","[1642723200000, 20930163211.171124]"
1093,"[1642809600000, 36508.799789830824]","[1642809600000, 692855029162.9379]","[1642809600000, 38296960033.31628]"
